In [1]:
import numpy as np
import h5py

In [2]:
bind = 0

mn_g_steps = {}

with h5py.File("../2025_06_04_double_check_cats/metadetect_2024-11-07.hdf5", "r") as fp:
    for mdet_step in fp.keys():
        print(mdet_step, flush=True)
        _g1 = fp[mdet_step][f"tomo_bin_{bind}"]["gauss_g_1"][:]
        print("read g1", flush=True)
        _g2 = fp[mdet_step][f"tomo_bin_{bind}"]["gauss_g_2"][:]
        print("read g2", flush=True)
        _w = fp[mdet_step][f"tomo_bin_{bind}"]["w"][:]
        print("read w", flush=True)
        mn_g_steps[mdet_step] = np.array([
            np.average(_g1, weights=_w),
            np.average(_g2, weights=_w)
        ])

        if mdet_step == "noshear":
            ra = fp[mdet_step][f"tomo_bin_{bind}"]["ra"][:]
            dec = fp[mdet_step][f"tomo_bin_{bind}"]["dec"][:]
            g1 = _g1.copy()
            g2 = _g2.copy()
            w = _w.copy()

deltag = 0.01

R11 = (
    mn_g_steps["1p"][0] - mn_g_steps["1m"][0]
) / 2 / deltag
R22 = (
    mn_g_steps["2p"][1] - mn_g_steps["2m"][1]
) / 2 / deltag

R = (R11 + R22) / 2.0


1m
read g1
read g2
read w
1p
read g1
read g2
read w
2m
read g1
read g2
read w
2p
read g1
read g2
read w
noshear
read g1
read g2
read w


In [3]:
def compute_shape_noise_h12(g1, g2, w, R):
    sumw = np.sum(w)
    sumw2 = np.sum(w**2)
    denom = (sumw * R)**2
    mn_g1 = np.average(g1, weights=w)
    numer_g1 = np.sum((w * (g1 - mn_g1))**2)

    mn_g2 = np.average(g2, weights=w)
    numer_g2 = np.sum((w * (g2 - mn_g2))**2)

    Neff = sumw**2 / sumw2

    return 0.5 * (numer_g1 / denom + numer_g2 / denom) * Neff

## Test 1-point Rotations

In [4]:
Neff = np.sum(w)**2 / np.sum(w**2)

rng = np.random.default_rng(seed=42)

g = g1 + 1j * g2
orig = np.sqrt(compute_shape_noise_h12(g1, g2, w, R))
raw = np.sqrt(
    0.5 * (
        np.mean((g1 - np.average(g1, weights=w))**2)
        + np.mean((g2 - np.average(g2, weights=w))**2)
    )
) / R

In [6]:
from mattspy import LokyParallel
import tqdm
import joblib

def _report(g1s, g2s, orig, Neff):
    print(
        "    n_samples|H12|meas:",
        len(g1s),
        orig / np.sqrt(Neff),
        np.sqrt(0.5 * (np.std(g1s)**2 + np.std(g2s)**2)),
        flush=True,
    )


def _measure(seed, g, R):
    _rng = np.random.default_rng(seed=seed)
    theta = _rng.uniform(size=w.shape[0]) * np.pi
    rot = np.exp(2j * theta)
    rg = g * rot
    rg1 = rg.real
    rg2 = rg.imag

    # assert not np.allclose(g1, rg1)
    # assert not np.allclose(g2, rg2)

    return (
        np.average(rg1, weights=w) / R,
        np.average(rg2, weights=w) / R,
    )


n_samples = 1000
seed = 56
rng = np.random.default_rng(seed=seed)
seeds = rng.uniform(low=1, high=1e10, size=n_samples).astype(np.int64)

with LokyParallel(n_jobs=2) as exc:
    g1s = []
    g2s = []
    for pr in tqdm.tqdm(
        exc(joblib.delayed(_measure)(seed, g, R) for seed in seeds),
        total=n_samples
    ):
        try:
            _g1, _g2 = pr.result()
        except Exception:
            pass
        else:
            g1s.append(_g1)
            g2s.append(_g2)

        if (len(g1s) > 0 and (len(g1s) % 10) == 0):
            _report(g1s, g2s, orig, Neff)

_report(g1s, g2s, orig, Neff)

  1%|          | 9/1000 [00:32<57:45,  3.50s/it]  

    n_samples|H12|meas: 10 4.875233211806958e-05 3.300279141225322e-05


  2%|▏         | 19/1000 [01:03<53:34,  3.28s/it]

    n_samples|H12|meas: 20 4.875233211806958e-05 4.4592835678300515e-05


  3%|▎         | 29/1000 [01:31<44:17,  2.74s/it]

...

    n_samples|H12|meas: 990 4.875233211806958e-05 4.8240207913357916e-05


100%|█████████▉| 999/1000 [48:17<00:02,  2.96s/it]

    n_samples|H12|meas: 1000 4.875233211806958e-05 4.815488098805658e-05


100%|██████████| 1000/1000 [48:20<00:00,  2.90s/it]

    n_samples|H12|meas: 1000 4.875233211806958e-05 4.815488098805658e-05
